In [ ]:
import SimpleITK as sitk

def save_array_to_nii(mask_array, nii_template_path, nii_save_path):

    sitk_image_object = sitk.ReadImage(nii_template_path)
    output_spacing = sitk_image_object.GetSpacing()
    output_direction = sitk_image_object.GetDirection()
    output_origin = sitk_image_object.GetOrigin()
#     print(output_spacing ,output_direction,output_origin)

    nrrd_output = sitk.GetImageFromArray(mask_array)
    nrrd_output.SetSpacing(output_spacing)
    nrrd_output.SetDirection(output_direction)
    nrrd_output.SetOrigin(output_origin)

    nrrdWriter = sitk.ImageFileWriter()
    nrrdWriter.SetFileName(nii_save_path)
    nrrdWriter.SetUseCompression(True)
    nrrdWriter.Execute(nrrd_output)
    print(nii_save_path ,'saved')

In [ ]:
import os
import glob
import SimpleITK as sitk
import numpy as np

# Input directory: original L4/L5 images in NIfTI format
img_L45_dir = 'videos/word/data_in_nii/img_in_nii_L4L5'

# Output directory: clipped images will be saved here
clipped_dir = 'videos/word/data_in_nii/img_in_nii_L4L5_clipped'

# Create the output directory if it does not already exist
os.makedirs(clipped_dir, exist_ok=True)

# Get the list of all .nii.gz files in the input directory
file_list = glob.glob(os.path.join(img_L45_dir, '*.nii.gz'))

for file_path in file_list:
    print("Processing:", file_path)
    
    # Read the image using SimpleITK
    img_sitk = sitk.ReadImage(file_path)
    img_array = sitk.GetArrayFromImage(img_sitk)
    print("Original range:", img_array.min(), img_array.max())
    
    # Clip the intensity values to the range [-30, 150]
    img_array_clipped = np.clip(img_array, -30, 150)
    
    # Construct the save path (same filename, new directory)
    filename = os.path.basename(file_path)
    save_path = os.path.join(clipped_dir, filename)
    
    # Save the clipped array back as a NIfTI file
    save_array_to_nii(img_array_clipped, file_path, save_path)


In [ ]:
import os
import glob
import SimpleITK as sitk


def downsample_slices_and_adjust_spacing(input_path, output_path, factor=2):
    """
    Downsample slices along the z-axis by a given factor and adjust the image spacing.

    Args:
        input_path (str): Path to the input image (.nii.gz or other SimpleITK-supported format).
        output_path (str): Path where the downsampled image will be saved.
        factor (int): Downsampling factor.
                      For example:
                          - factor=2 → keep every 2nd slice
                          - factor=3 → keep every 3rd slice
    """
    # Read the image
    img = sitk.ReadImage(input_path)
    arr = sitk.GetArrayFromImage(img)  # Shape: (slices, height, width)

    # Downsample along the slice (z) axis
    arr_downsampled = arr[::factor, :, :]

    # Adjust z-axis spacing (multiply by the downsampling factor)
    spacing = list(img.GetSpacing())
    spacing[2] *= factor

    # Create a new image from the downsampled array
    img_downsampled = sitk.GetImageFromArray(arr_downsampled)
    img_downsampled.SetSpacing(spacing)
    img_downsampled.SetOrigin(img.GetOrigin())
    img_downsampled.SetDirection(img.GetDirection())

    # Save the downsampled image
    sitk.WriteImage(img_downsampled, output_path)
    print(f"Saved downsampled: {output_path}, shape: {arr_downsampled.shape}, spacing: {spacing}, factor: {factor}")


# Dataset name
dataset = 'word'

# Input directories
img_dir = f'videos/{dataset}/data_in_nii/img_in_nii_L4L5_clipped'
label_dir = f'videos/{dataset}/data_in_nii/label_in_nii_L4L5'

# Output directories
output_img_dir = f'videos/{dataset}/data_in_nii/img_in_nii_L4L5_downsampled'
output_label_dir = f'videos/{dataset}/data_in_nii/label_in_nii_L4L5_downsampled'

# Create output directories if they don’t exist
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Process all image files
for img_path in glob.glob(os.path.join(img_dir, '*.nii.gz')):
    out_path = os.path.join(output_img_dir, os.path.basename(img_path))
    downsample_slices_and_adjust_spacing(img_path, out_path)

# Process all label files
for label_path in glob.glob(os.path.join(label_dir, '*.nii.gz')):
    out_path = os.path.join(output_label_dir, os.path.basename(label_path))
    downsample_slices_and_adjust_spacing(label_path, out_path)
